xml version: 5dfa9a5

In [20]:
xmlFile = 'MTM2_Telemetry.xml'
partialxml = '''
Event:
SummaryState
AssemblyInPosition
AppliedSettingMatchStart
ErrorCode
Telemetry:
MirrorPositionMeasured
AxialForcesMeasured
TangentForcesMeasured
TemperatureMeasured
ZenithAngleMeasured
AxialActuatorSteps
TangentActuatorSteps
AxialAcutatorEncoder
TangentActuatorEncoder'''
partialxml = partialxml.lower()

In [3]:
import numpy as np
from astropy.time import Time
from datetime import timedelta, datetime
from lsst_efd_client import EfdClient

import pandas as pd

In [27]:
client = EfdClient('summit_efd')
csc_index = 1

#start = Time('2020-11-03T13:41:30') #this is UTC
#end = start+timedelta(days=1)
end = Time(datetime.now())
end = Time('2020-11-05T18:03:30') #this is UTC
start = end - timedelta(seconds=5)
#import astropy
#astropy.utils.data.clear_download_cache()

In [28]:
fid = open(xmlFile)

topics = dict()
params = dict()
counts = dict()
lastT = dict()
values= dict()
nline = 0
for line in fid:
    if '<EFDB_Topic>' in line:
        validTopic = False
        i = line.find('<EFDB')
        topic = line[i+12:-14]
        t1, t2 = topic.split('_',1)
        if 'partialxml' in globals() and t2.lower() in partialxml:
            validTopic = True
            dfa = await client.select_time_series('lsst.sal.%s.%s'%(t1,t2), '*', 
                                     (start-timedelta(seconds=37)).tai, (end-timedelta(seconds=37)).tai, csc_index)
    elif '<EFDB_Name>' in line:
        i = line.find('<EFDB')
        param = line[i+11:-13]
        if validTopic:
            nline+=1
            idx = '%d'%nline
    elif '<Count>' in line:
        i = line.find('<C')
        count = line[i+7:-9]
        
        if validTopic:
            print(topic, param,count)
            topics[idx] = topic
            params[idx] = param
            counts[idx] = count
            c = int(count)
            try:
                lastT[idx] = pd.to_datetime(dfa.private_sndStamp[-1], unit='s')
                if c == 1:
                    values[idx] = dfa[param][-1]
                else:
                    aa = [dfa['%s%d'%(param,i)][-1] for i in range(c)]
                    values[idx] = np.mean(aa)
                    if np.isnan(values[idx]):
                        print(aa)
            except:
                lastT[idx] = np.nan
                values[idx] = np.nan
print(nline)
fid.close()

MTM2_mirrorPositionMeasured xTilt 1
MTM2_mirrorPositionMeasured yTilt 1
MTM2_mirrorPositionMeasured piston 1
MTM2_mirrorPositionMeasured xPosition 1
MTM2_mirrorPositionMeasured yPosition 1
MTM2_mirrorPositionMeasured thetaZPosition 1
MTM2_axialForcesMeasured axialForceMeasured 72
MTM2_tangentForcesMeasured tangentLink0DegForceMeasured 1
MTM2_tangentForcesMeasured tangentLink60DegForceMeasured 1
MTM2_tangentForcesMeasured tangentLink120DegForceMeasured 1
MTM2_tangentForcesMeasured tangentLink180DegForceMeasured 1
MTM2_tangentForcesMeasured tangentLink240DegForceMeasured 1
MTM2_tangentForcesMeasured tangentLink300DegForceMeasured 1
MTM2_zenithAngleMeasured zenithAngleMeasured 1
14


In [29]:
frame = {'Topic': topics, 'Name': params,
         'Count': counts, 'Last Timestamp (UTC)': lastT,
         'value (average)': values}
d = pd.DataFrame(frame)

In [30]:
d

,Topic,Name,Count,Last Timestamp (UTC),value (average)
1,MTM2_mirrorPositionMeasured,xTilt,1,2020-11-05 18:03:29.984337091,4.071077e-07
2,MTM2_mirrorPositionMeasured,yTilt,1,2020-11-05 18:03:29.984337091,4.133858e-07
3,MTM2_mirrorPositionMeasured,piston,1,2020-11-05 18:03:29.984337091,8.385593e-02
4,MTM2_mirrorPositionMeasured,xPosition,1,2020-11-05 18:03:29.984337091,-2.148263e+01
5,MTM2_mirrorPositionMeasured,yPosition,1,2020-11-05 18:03:29.984337091,-1.611617e-02
6,MTM2_mirrorPositionMeasured,thetaZPosition,1,2020-11-05 18:03:29.984337091,-6.041731e-06
7,MTM2_axialForcesMeasured,axialForceMeasured,72,2020-11-05 18:03:29.976671696,2.163279e+02
8,MTM2_tangentForcesMeasured,tangentLink0DegForceMeasured,1,2020-11-05 18:03:29.980070591,4.588767e+01
9,MTM2_tangentForcesMeasured,tangentLink60DegForceMeasured,1,2020-11-05 18:03:29.980070591,-1.191893e+02
10,MTM2_tangentForcesMeasured,tangentLink120DegForceMeasured,1,2020-11-05 18:03:29.980070591,1.443249e+01


In [31]:
d[np.isnan(d['value (average)'])]

,Topic,Name,Count,Last Timestamp (UTC),value (average)
